In [1]:
 pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=073c477c19602375c55a2c7bc5945c688221371cc1254c9c99e8cbe9d168d687
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
from deepface import DeepFace
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.optimizers import Adam
import cv2

2025-07-01 13:41:37.739380: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751377297.932683      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751377297.991076      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


25-07-01 13:41:49 - Directory /root/.deepface has been created
25-07-01 13:41:49 - Directory /root/.deepface/weights has been created


In [3]:
# Define paths (adjust according to your dataset)
TRAIN_DATASET_PATH = '/kaggle/input/fully-train/train'
VAL_DATASET_PATH = "/kaggle/input/validation/val"
IDENTITY_FOLDERS_TRAIN = [f for f in os.listdir(TRAIN_DATASET_PATH) if os.path.isdir(os.path.join(TRAIN_DATASET_PATH, f))]
IDENTITY_FOLDERS_VAL = [f for f in os.listdir(VAL_DATASET_PATH) if os.path.isdir(os.path.join(VAL_DATASET_PATH, f))]

In [4]:
# Function to load clear and distorted images and extract embeddings
def load_and_embed_images(folder_path):
    clear_embeddings = []
    distorted_embeddings = []
    labels = []
    
    # Assuming VGG-Face produces 4096-dimensional embeddings
    EMBEDDING_DIM = 4096 

    for identity in os.listdir(folder_path):
        identity_full_path = os.path.join(folder_path, identity)
        if os.path.isdir(identity_full_path):
            # Load clear reference image (assuming one clear image per folder)
            clear_images = [f for f in os.listdir(identity_full_path) if f.lower().endswith(('.jpg', '.jpeg', '.png')) and not os.path.isdir(os.path.join(identity_full_path, f))]
            
            # Process clear images
            if clear_images:
                clear_img_path = os.path.join(identity_full_path, clear_images[0])
                try:
                    clear_embedding_dicts = DeepFace.represent(clear_img_path, model_name="VGG-Face", enforce_detection=False)
                    if clear_embedding_dicts:
                        clear_embedding = clear_embedding_dicts[0]['embedding']
                        clear_embeddings.append(clear_embedding)
                        labels.append(identity) # Label for the clear image
                    else:
                        print(f"No embeddings found for clear image {clear_img_path}")
                except Exception as e:
                    print(f"Error processing clear image {clear_img_path}: {e}")
            else:
                print(f"No clear images found in {identity_full_path}")

            # Load distorted images from the nested distorted folder
            distorted_folder = os.path.join(identity_full_path, "distortion")
            if os.path.isdir(distorted_folder):
                for dist_img_name in os.listdir(distorted_folder):
                    dist_img_path = os.path.join(distorted_folder, dist_img_name)
                    if dist_img_name.lower().endswith(('.jpg', '.jpeg', '.png')): # Ensure it's an image file
                        try:
                            dist_embedding_dicts = DeepFace.represent(dist_img_path, model_name="VGG-Face", enforce_detection=False)
                            if dist_embedding_dicts:
                                dist_embedding = dist_embedding_dicts[0]['embedding']
                                distorted_embeddings.append(dist_embedding)
                                labels.append(identity) # Label for the distorted image
                            else:
                                print(f"No embeddings found for distorted image {dist_img_path}")
                        except Exception as e:
                            print(f"Error processing distorted image {dist_img_path}: {e}")
            else:
                print(f"Distorted folder not found for identity {identity} at {distorted_folder}")
    
    # Convert to numpy arrays. Ensure they are 2D and have the correct embedding dimension.
    # If empty, create an empty array with the correct second dimension for concatenation.
    if not clear_embeddings:
        clear_embeddings_np = np.empty((0, EMBEDDING_DIM))
    else:
        clear_embeddings_np = np.array(clear_embeddings)
        if clear_embeddings_np.ndim == 1: # If only one embedding was found, it might be 1D
            clear_embeddings_np = clear_embeddings_np.reshape(1, -1)
        # Ensure the second dimension matches EMBEDDING_DIM, if not, there's an issue with DeepFace output
        if clear_embeddings_np.shape[1] != EMBEDDING_DIM:
            print(f"Warning: Clear embeddings have unexpected dimension {clear_embeddings_np.shape[1]}, expected {EMBEDDING_DIM}")
            # Handle this case, e.g., by filtering or padding, or raising an error.
            # For now, we'll proceed, but this might lead to further errors if not handled.

    if not distorted_embeddings:
        distorted_embeddings_np = np.empty((0, EMBEDDING_DIM))
    else:
        distorted_embeddings_np = np.array(distorted_embeddings)
        if distorted_embeddings_np.ndim == 1: # If only one embedding was found, it might be 1D
            distorted_embeddings_np = distorted_embeddings_np.reshape(1, -1)
        # Ensure the second dimension matches EMBEDDING_DIM
        if distorted_embeddings_np.shape[1] != EMBEDDING_DIM:
            print(f"Warning: Distorted embeddings have unexpected dimension {distorted_embeddings_np.shape[1]}, expected {EMBEDDING_DIM}")
            
    return clear_embeddings_np, distorted_embeddings_np, labels

In [5]:
# Siamese Network for learning similarity
def create_siamese_network(input_shape):
    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)
    
    # Shared layers for feature extraction
    # Define the embedding network once and reuse it for both inputs
    embedding_network = tf.keras.Sequential([
        Dense(128, activation="relu"),
        Dense(64, activation="relu")
    ])
    
    processed_a = embedding_network(input_a)
    processed_b = embedding_network(input_b)
    
    # Calculate Euclidean distance
    distance = Lambda(lambda tensors: tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1, keepdims=True))([processed_a, processed_b])
    
    siamese_model = Model(inputs=[input_a, input_b], outputs=distance)
    return siamese_model


In [6]:
def contrastive_loss(y_true, y_pred):
    margin = 1.0  # Define the margin hyperparameter
    y_true = tf.cast(y_true, tf.float32)  # Ensure y_true is float32
    square_pred = tf.square(y_pred)
    margin_square = tf.square(tf.maximum(margin - y_pred, 0.0))
    return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [7]:
# Generate pairs for training
def generate_pairs(clear_embeddings, distorted_embeddings, labels):
    pairs = []
    pair_labels = []

    # It's crucial that all_embeddings and all_labels are correctly aligned.
    # If labels are associated with individual embeddings, then concatenate them.
    # Assuming labels list contains an entry for each clear and each distorted embedding.
    # If labels are per identity, then this needs to be adjusted.
    # For simplicity, let's assume labels are per identity and we need to map them to embeddings.
    
    # A more robust way to generate pairs would be to iterate through identities
    # and then pick clear/distorted images from that identity for positive pairs,
    # and images from other identities for negative pairs.

    # Let's refine the pair generation based on the assumption that `labels`
    # contains the identity for each corresponding embedding in `clear_embeddings`
    # and `distorted_embeddings`.
    
    # Create a mapping from identity to list of embeddings for that identity
    identity_to_embeddings = {}
    for i, emb in enumerate(clear_embeddings):
        identity = labels[i] # Assuming labels are aligned with clear_embeddings first
        if identity not in identity_to_embeddings:
            identity_to_embeddings[identity] = []
        identity_to_embeddings[identity].append(emb)
    
    # Adjusting for distorted embeddings and their labels.
    # The original `load_and_embed_images` appends labels for both clear and distorted images.
    # So, `labels` will contain `len(clear_embeddings) + len(distorted_embeddings)` entries.
    # We need to ensure `all_embeddings` and `all_labels` are correctly formed.

    # Let's re-think `all_embeddings` and `all_labels` based on the `load_and_embed_images` output.
    # `load_and_embed_images` appends `identity` to `labels` for each embedding it successfully extracts.
    # So, `labels` will contain the identity for each embedding in `clear_embeddings` followed by `distorted_embeddings`.
    
    # This means `labels` is already a combined list of labels for all embeddings.
    # We need to ensure `all_embeddings` is also correctly combined.
    
    # Check if clear_embeddings or distorted_embeddings are empty before concatenation
    if clear_embeddings.size == 0 and distorted_embeddings.size == 0:
        print("Warning: No embeddings available to generate pairs.")
        return np.array([]).reshape(0, 2, EMBEDDING_DIM), np.array([]) # Return empty arrays with correct shape

    # Ensure concatenation works even if one is empty but the other is not
    if clear_embeddings.size == 0:
        all_embeddings = distorted_embeddings
    elif distorted_embeddings.size == 0:
        all_embeddings = clear_embeddings
    else:
        all_embeddings = np.concatenate([clear_embeddings, distorted_embeddings], axis=0)
    
    # The `labels` list from `load_and_embed_images` already contains labels for all extracted embeddings.
    # So, `all_labels` is simply the `labels` list.
    all_labels = labels 

    # Create a dictionary to group embeddings by identity
    embeddings_by_identity = {}
    for i, emb in enumerate(all_embeddings):
        current_label = all_labels[i]
        if current_label not in embeddings_by_identity:
            embeddings_by_identity[current_label] = []
        embeddings_by_identity[current_label].append(emb)

    # Generate pairs
    for i, (emb1, label1) in enumerate(zip(all_embeddings, all_labels)):
        # Positive pair (same identity)
        # Pick another embedding from the same identity
        same_identity_embeddings = [e for e in embeddings_by_identity[label1] if not np.array_equal(e, emb1)]
        if same_identity_embeddings:
            pos_emb = same_identity_embeddings[np.random.choice(len(same_identity_embeddings))]
            pairs.append([emb1, pos_emb])
            pair_labels.append(1)
        
        # Negative pair (different identity)
        # Pick an embedding from a different identity
        other_identities = [l for l in embeddings_by_identity if l != label1]
        if other_identities:
            neg_label = np.random.choice(other_identities)
            neg_emb = embeddings_by_identity[neg_label][np.random.choice(len(embeddings_by_identity[neg_label]))]
            pairs.append([emb1, neg_emb])
            pair_labels.append(0)
    
    # Convert to numpy arrays
    if not pairs:
        # If no pairs could be generated, return empty arrays with the expected shape
        return np.array([]).reshape(0, 2, EMBEDDING_DIM), np.array([])
    
    return np.array(pairs), np.array(pair_labels)



In [8]:
# Main function
def main():
    # Load training data
    print("Loading training data and generating embeddings...")
    clear_train_embeddings, distorted_train_embeddings, train_labels = load_and_embed_images(TRAIN_DATASET_PATH)
    print(f"Train: Clear embeddings shape: {clear_train_embeddings.shape}, Distorted embeddings shape: {distorted_train_embeddings.shape}")
    
    # Load validation data
    print("Loading validation data and generating embeddings...")
    clear_val_embeddings, distorted_val_embeddings, val_labels = load_and_embed_images(VAL_DATASET_PATH)
    print(f"Validation: Clear embeddings shape: {clear_val_embeddings.shape}, Distorted embeddings shape: {distorted_val_embeddings.shape}")

    # Generate pairs for training and validation
    print("Generating training pairs...")
    train_pairs, train_pair_labels = generate_pairs(clear_train_embeddings, distorted_train_embeddings, train_labels)
    print(f"Generated {len(train_pairs)} training pairs.")

    print("Generating validation pairs...")
    val_pairs, val_pair_labels = generate_pairs(clear_val_embeddings, distorted_val_embeddings, val_labels)
    print(f"Generated {len(val_pairs)} validation pairs.")

    # Check if there are enough pairs to train
    if len(train_pairs) == 0:
        print("Error: No training pairs generated. Cannot train the model.")
        return
    if len(val_pairs) == 0:
        print("Warning: No validation pairs generated. Model will be trained without validation.")
        validation_data = None
    else:
        validation_data = ([val_pairs[:, 0], val_pairs[:, 1]], val_pair_labels)

    # Create and compile Siamese Network
    # The input_shape should be the dimension of a single embedding (e.g., 4096,)
    # We need to ensure clear_train_embeddings is not empty to get its shape.
    if clear_train_embeddings.shape[0] > 0:
        input_shape = clear_train_embeddings.shape[1:] # Should be (4096,)
    elif distorted_train_embeddings.shape[0] > 0:
        input_shape = distorted_train_embeddings.shape[1:] # Should be (4096,)
    else:
        print("Error: No embeddings found to determine input shape for the Siamese network.")
        return

    print(f"Siamese Network input shape: {input_shape}")
    siamese_model = create_siamese_network(input_shape)
    siamese_model.compile(loss=contrastive_loss, optimizer=Adam(learning_rate=0.0001))
    
    # Train the model
    print("Training the Siamese model...")
    siamese_model.fit(
        [train_pairs[:, 0], train_pairs[:, 1]], train_pair_labels,
        validation_data=validation_data,
        epochs=20, batch_size=32, verbose=1
    )
    
    # Save model
    siamese_model.save("siamese_face_matching.h5")
    print("Model saved as siamese_face_matching.h5")


In [9]:
if __name__ == "__main__":
    main()

Loading training data and generating embeddings...


I0000 00:00:1751377313.007098      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


25-07-01 13:41:53 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:06<00:00, 87.7MB/s]
I0000 00:00:1751377323.385443      19 cuda_dnn.cc:529] Loaded cuDNN version 90300


Train: Clear embeddings shape: (877, 4096), Distorted embeddings shape: (13482, 4096)
Loading validation data and generating embeddings...
Validation: Clear embeddings shape: (250, 4096), Distorted embeddings shape: (2954, 4096)
Generating training pairs...
Generated 28718 training pairs.
Generating validation pairs...
Generated 6408 validation pairs.
Siamese Network input shape: (4096,)
Training the Siamese model...
Epoch 1/20


I0000 00:00:1751382127.850301      74 service.cc:148] XLA service 0x7a4608918930 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751382127.850977      74 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751382128.110980      74 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


898/898 [==============================] - 7s 4ms/step - loss: 0.2704 - val_loss: 0.2508
Epoch 2/20
898/898 [==============================] - 3s 3ms/step - loss: 0.2187 - val_loss: 0.2449
Epoch 3/20
898/898 [==============================] - 3s 3ms/step - loss: 0.2000 - val_loss: 0.2424
Epoch 4/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1861 - val_loss: 0.2423
Epoch 5/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1749 - val_loss: 0.2418
Epoch 6/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1655 - val_loss: 0.2426
Epoch 7/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1571 - val_loss: 0.2449
Epoch 8/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1499 - val_loss: 0.2460
Epoch 9/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1436 - val_loss: 0.2472
Epoch 10/20
898/898 [==============================] - 3s 3ms/step - loss: 0.1380 - val_loss: 0.2477
Epoch 11/2